In [ ]:
import torch
from fourier_scaffold import (
    FourierScaffold,
    HammardSharpening,
    HammardShift,
    HammardShiftMatrix,
    HammardSharpening,
    ContractionSharpening,
    GaussianFourierSmoothing,
    GuassianFourierSmoothingMatrix,
)
from graph_utils import fourier_plot_probabilities_complex

In [ ]:
shapes = [[5, 5], [7, 7]]
device = "cuda"
scaffold = FourierScaffold(
    shapes=torch.tensor(shapes),
    D=1000**2,
    smoothing=GaussianFourierSmoothing(kernel_radii=[10] * 2, kernel_sigmas=[0.4] * 2),
    device=device,
)

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(subplot_kw={"projection": "polar"})
ax.set_ylim(0,1)
fourier_plot_probabilities_complex(scaffold, ax)
plt.show()

In [ ]:
shapes = [[5, 5], [7, 7]]
device = "cuda"
scaffold = FourierScaffold(
    shapes=torch.tensor(shapes),
    D=1000,
    smoothing=GuassianFourierSmoothingMatrix(kernel_radii=[10] * 2, kernel_sigmas=[0.4] * 2),
    sharpening=ContractionSharpening(2),
    shift=HammardShiftMatrix(),
    device=device,
    representation='matrix'
)


In [ ]:
import torch
from graph_utils import fourier_plot_probabilities_complex

# Experiments to test the approximation of the FourierScaffold under different sharpening methods and distributions

from fourier_scaffold import (
    FourierScaffold,
    FourierScaffoldDebug,
    HammardSharpening,
    ContractionSharpening,
    GaussianFourierSmoothing,
    GuassianFourierSmoothingMatrix,
    HammardShiftMatrix,
    calculate_padding,
)
import math
import matplotlib.pyplot as plt

device = "cuda"
shapes = torch.tensor([(5, 5), (7, 7)], device=device)
dim_sizes = [int(shapes[:, dim].prod().item()) for dim in range(shapes.shape[1])]


def zero():
    return torch.zeros(*dim_sizes, device=device)


def uniform():
    t = torch.ones_like(zero())
    return t / t.sum()


def degenerate():
    t = zero()
    t[tuple([0] * shapes.shape[1])] = 1
    return t


def gaussian(sigma=1):
    t = degenerate()
    kernel_size = 2 * max(10, 3 * math.ceil(sigma)) + 1
    x = torch.arange(kernel_size, device=device) - kernel_size // 2
    y = torch.arange(kernel_size, device=device) - kernel_size // 2
    x, y = torch.meshgrid(x, y)
    kernel = torch.exp(-(x**2 + y**2) / (2 * sigma**2))
    kernel = kernel / kernel.sum()

    x_padding = calculate_padding(kernel_size, kernel.shape[0], 1)
    y_padding = calculate_padding(kernel_size, kernel.shape[1], 1)

    padded = torch.nn.functional.pad(
        t.unsqueeze(0).unsqueeze(0),
        y_padding + x_padding,
        mode="circular",
    )

    convoluted = torch.nn.functional.conv2d(
        input=padded, weight=kernel.unsqueeze(0).unsqueeze(0)
    )

    return convoluted.squeeze(0).squeeze(0)


def bimodal():
    t = zero()
    index = [0] * shapes.shape[1]
    for i, size in enumerate(dim_sizes):
        index[i] = size // 2

    t[tuple([0] * shapes.shape[1])] = 0.5
    t[tuple(index)] = 0.5
    return t

In [ ]:
distributions = [
    ("degenerate", degenerate()),
    ("uniform", uniform()),
    ("gaussian 0.5", gaussian(0.5)),
    ("gaussian 1", gaussian(1)),
    ("gaussian 2", gaussian(2)),
    ("bimodal", bimodal()),
]

scaffold = FourierScaffold(
    shapes,
    D=10**6,
    sharpening=HammardSharpening(2),
    smoothing=GaussianFourierSmoothing(
        kernel_radii=[10, 10], kernel_sigmas=[1, 1]
    ),  # doesnt matter just so it runs
    device=device,
)
# scaffold = FourierScaffold(
#     shapes,
#     D=10**3,
#     sharpening=ContractionSharpening(2),
#     shift=HammardShiftMatrix(),
#     smoothing=GuassianFourierSmoothingMatrix(
#         kernel_radii=[10, 10], kernel_sigmas=[1, 1]
#     ),  # doesnt matter just so it runs
#     device=device,
#     representation='matrix'
# )
scaffold_debug = FourierScaffoldDebug(shapes, device=device)


def run_test(
    distribution: torch.Tensor,
    scaffold: FourierScaffold,
    scaffold_debug: FourierScaffoldDebug,
):
    scaffold.g = scaffold.encode_probability(distribution)
    scaffold_debug.ptensor = distribution

    print(scaffold.g)
    print(scaffold_debug.ptensor)

    estimated = scaffold.get_all_probabilities().flatten()
    true = scaffold_debug.ptensor.flatten()
    print("encoding similarity:", torch.cosine_similarity(true, estimated, dim=0))

    scaffold.sharpen()
    scaffold_debug.sharpen()

    print(scaffold.g)
    print(scaffold_debug.ptensor)

    estimated = scaffold.get_all_probabilities().flatten()
    true = scaffold_debug.ptensor.flatten()
    print(
        "similarity after sharpning:", torch.cosine_similarity(true, estimated, dim=0)
    )


for name, distribution in distributions:
    print(f" ----------------------- running test: {name} --------------------")
    run_test(distribution, scaffold, scaffold_debug)